# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Corrected file path
file_path = "/Users/janlelie/Desktop/Git Hub Repositories/python-api-challenge/WeatherPy/output_data/cities.csv"

# Load the CSV file
city_data_df = pd.read_csv(file_path)

# Display the first few rows to verify
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mossel bay,-34.1831,22.1460,13.96,79,44,3.08,ZA,1732926541
1,1,bredasdorp,-34.5322,20.0403,15.72,85,72,1.83,ZA,1732926543
2,2,afaahiti,-17.7500,-149.2833,25.98,79,82,6.63,PF,1732926546
3,3,margate,51.3813,1.3862,8.16,82,13,7.72,GB,1732926547
4,4,port-aux-francais,-49.3500,70.2167,4.08,87,100,6.25,TF,1732926549


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
import hvplot.pandas
import geoviews as gv
from geoviews import opts
import holoviews as hv

# Enable Holoviews extension for bokeh
hv.extension('bokeh')

# Configure the map plot
city_map = city_data_df.hvplot.points(
    'Lng', 'Lat',           # Longitude and Latitude for plotting
    geo=True,               # Enable geospatial plotting
    size='Humidity',        # Size of the points based on humidity
    color='Humidity',       # Color the points by humidity
    alpha=0.6,              # Add transparency for better visualization
    tiles='OSM',            # OpenStreetMap tiles for the base map
    frame_width=800,        # Map width
    frame_height=600,       # Map height
    title="City Humidity Map"
)

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria
ideal_cities_df = city_data_df[
    (city_data_df["Max Temp"] > 21) & 
    (city_data_df["Max Temp"] < 27) & 
    (city_data_df["Wind Speed"] < 4.5) & 
    (city_data_df["Cloudiness"] == 0)
]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
217,217,lazaro cardenas,17.9583,-102.2000,25.50,83,0,2.16,MX,1732926882
267,267,cabo san lucas,22.8909,-109.9124,25.17,69,0,2.57,MX,1732926888
280,280,san sebastian de la gomera,28.0916,-17.1133,26.20,19,0,1.03,ES,1732926981
313,313,porbandar,21.6422,69.6093,22.37,39,0,3.59,IN,1732927032
413,413,port augusta,-32.5000,137.7667,24.62,46,0,4.17,AU,1732927189


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df
# Include city, country, coordinates, and humidity
hotel_df = ideal_cities_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
217,lazaro cardenas,MX,17.9583,-102.2000,83,
267,cabo san lucas,MX,22.8909,-109.9124,69,
280,san sebastian de la gomera,ES,28.0916,-17.1133,19,
313,porbandar,IN,21.6422,69.6093,39,
413,port augusta,AU,-32.5000,137.7667,46,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
import time

# Set parameters to search for a hotel
radius = 10000  # 10,000 meters (10 km)
params = {
    "categories": "accommodation.hotel",  # Search for hotels
    "limit": 1,  # Retrieve only the first result
    "apiKey": geoapify_key  # Your Geoapify API key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found"
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

    # Pause to avoid exceeding the rate limit
    time.sleep(1)

# Display sample data
hotel_df

Starting hotel search
lazaro cardenas - nearest hotel: Hotel Sol del Pacífico
cabo san lucas - nearest hotel: Comfort Rooms
san sebastian de la gomera - nearest hotel: Hotel Villa Gomera
porbandar - nearest hotel: Toran Tourist Bungalow
port augusta - nearest hotel: Hotel Commonwealth
saint-laurent-du-maroni - nearest hotel: La Tentiaire
sabancuy - nearest hotel: Hotel plaza sabancuy
sur - nearest hotel: Sur Hotel
badiyah - nearest hotel: Al Areen Hotel Apartments
grand gaube - nearest hotel: Hotel Lux- Grand Gaube


,City,Country,Lat,Lng,Humidity,Hotel Name
217,lazaro cardenas,MX,17.9583,-102.2000,83,Hotel Sol del Pacífico
267,cabo san lucas,MX,22.8909,-109.9124,69,Comfort Rooms
280,san sebastian de la gomera,ES,28.0916,-17.1133,19,Hotel Villa Gomera
313,porbandar,IN,21.6422,69.6093,39,Toran Tourist Bungalow
413,port augusta,AU,-32.5000,137.7667,46,Hotel Commonwealth
461,saint-laurent-du-maroni,GF,5.5039,-54.0289,94,La Tentiaire
504,sabancuy,MX,18.9667,-91.1833,77,Hotel plaza sabancuy
541,sur,OM,22.5667,59.5289,70,Sur Hotel
543,badiyah,OM,22.4500,58.8000,75,Al Areen Hotel Apartments
551,grand gaube,MU,-20.0064,57.6608,78,Hotel Lux- Grand Gaube


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    'Lng', 'Lat',               # Longitude and Latitude for plotting
    geo=True,                   # Enable geospatial plotting
    size='Humidity',            # Size of the points based on humidity
    color='Humidity',           # Color the points by humidity
    alpha=0.7,                  # Add transparency for better visualization
    tiles='OSM',                # OpenStreetMap tiles for the base map
    frame_width=800,            # Map width
    frame_height=600,           # Map height
    hover_cols=['Hotel Name', 'Country'],  # Add hover info for hotel name and country
    title="Hotel Map with Humidity"
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,Hotel Name,Country)